In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import pandas as pd
import seaborn as sns
sns.set()
import pandas as pd
import gc
gc.collect()
%matplotlib inline
import matplotlib.pyplot as plt
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')
from tqdm import tqdm

from pyhive import presto
import requests
from requests.auth import HTTPBasicAuth
import pandas as pd
requests.packages.urllib3.disable_warnings()
import matplotlib.pyplot as plt
import datetime as dt
import numpy as np
import gc
gc.collect()
gc.collect()
import gzip
tqdm.pandas()

/home/ronak/anaconda3/envs/customer_v1/lib/python3.7/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [10]:
file_name='1582100338-1582100338-orderTimerRendered-20200218-0-0.json.gz'
# file_name='1582100338-1582100338-orderTimerRendered-20200218-1-0.json.gz'
with gzip.open(file_name) as f:
    order_timer_rendered_raw=pd.read_json(file_name,lines=True)


In [11]:
order_timer_rendered_raw.shape

(1000001, 5)

In [12]:
import gzip
from datetime import datetime
def parse_order_time_rendered(file_name):
    def parse_time(x):
        return  datetime.strptime(str(x), '%Y%m%d%H%M%S')
    def get_captain_identity(profile_payload):
        try:
            identity_index=[i for i, s in enumerate(profile_payload) if 'identity' in s][0]
        except:
            return None
        captain_identity=list(profile_payload.values())[identity_index]
        return captain_identity

    def get_order_identity(event_prpos_payload):
        try:
            identity_index=[i for i, s in enumerate(event_prpos_payload) if 'orderId' in s][0]
        except:
            return None
        captain_identity=list(event_prpos_payload.values())[identity_index]
        return captain_identity
    def get_property_value_for_col(df,source_column,target_column,property_key):
        def get_property(payload,property_key):
            try:
                identity_index=[i for i, s in enumerate(payload) if property_key in s][0]
            except:
                return None
            value=list(payload.values())[identity_index]
            return value
        df.loc[:,target_column]=df[source_column].progress_apply(lambda x: get_property(x,property_key))
        return df

    order_timer_rendered_raw=None
    with gzip.open(file_name) as f:
        order_timer_rendered_raw=pd.read_json(file_name,lines=True)
    order_timer_rendered_raw['time']=order_timer_rendered_raw['ts'].progress_apply(parse_time)
    order_timer_rendered_raw.loc[:,'captain_id']=order_timer_rendered_raw['profile'].progress_apply(get_captain_identity)
    order_timer_rendered_raw.loc[:,'order_id']=order_timer_rendered_raw['eventProps'].progress_apply(get_order_identity)
    
    source_column='deviceInfo'
    target_column='app_version'
    property_key='appVersion'
    order_timer_rendered_raw=get_property_value_for_col(order_timer_rendered_raw,source_column,target_column,property_key)

    return order_timer_rendered_raw[['eventName','time','captain_id','order_id','app_version']]

In [13]:
order_timer_rendered=parse_order_time_rendered(file_name)

100%|██████████| 1000001/1000001 [00:03<00:00, 306201.76it/s]


In [14]:
order_timer_rendered.to_csv('orderTimeRendered_20200218-0-0.csv',index=False)
# order_timer_rendered.to_csv('orderTimeRendered_20200218-1-0.csv',index=False)

In [15]:
order_timer_rendered.shape

(1000001, 5)

In [16]:
order_timer_files=[
    'orderTimeRendered_20200218-1-0.csv',
    'orderTimeRendered_20200218-0-0.csv'
]

In [17]:
df_s=[]
for file in order_timer_files:
    df_s.append(pd.read_csv(file))

In [22]:
# df_s

In [18]:
pd.concat(df_s).to_csv('orderTimeRendered_20200218.csv',index=False)

In [24]:
order_timer_rendered_raw.time.min()
order_timer_rendered_raw.time.max()

AttributeError: 'DataFrame' object has no attribute 'time'

#### Scratchpad

In [6]:
with gzip.open(file_name) as f:
    order_timer_rendered_raw=pd.read_json(file_name,lines=True)


In [7]:
order_timer_rendered_raw.head(2)

,eventName,ts,eventProps,profile,deviceInfo
0,orderTimerRendered,20200218103007,"{'area': '----', 'cluster': 'Rasulpura', 'city...",{'all_identities': ['5dca7eba010c5a4601fbfb7f'...,"{'osVersion': '8.1.0', 'sdkVersion': '30601', ..."
1,orderTimerRendered,20200218103009,"{'area': '----', 'cluster': 'Hazrat ganj', 'ci...",{'all_identities': ['5d2823c406ee2c37f1f3937e'...,"{'osVersion': '8.1.0', 'sdkVersion': '30601', ..."


In [77]:
order_timer_rendered_raw['eventProps'].iloc[2]
get_order_identity(order_timer_rendered_raw['eventProps'].iloc[2])

{'area': '----',
 'cluster': '',
 'city': 'Bangalore',
 'Device rooted': 'false',
 'regCity': 'Bangalore',
 'CT Source': 'Mobile',
 'latitude': 12.711327,
 'longitude': 77.699585,
 'CT Latitude': 12.711327,
 'CT Longitude': 77.699585,
 'occurred_date': '$D_1581397311',
 'hhmmss': '103151',
 'orderId': '5e4235036735ce3fc338bc39',
 'yyyymmdd': '20200211',
 'networkspeed': '-1.0',
 'CT App Version': '3.10.2',
 'epoch': '1581397311990'}

'5e4235036735ce3fc338bc39'

In [74]:
eventPropsPayload=order_timer_rendered_raw['eventProps'].iloc[2]
id_index=[i for i, s in enumerate(eventPropsPayload) if 'orderId' in s][0]
list(eventPropsPayload.values())[id_index]

'5e4235036735ce3fc338bc39'

In [31]:
profile_payload=order_timer_rendered_raw['profile'].iloc[2]
profile_payload
["identity" in x for i,x in enumerate(profile_payload)]

{'all_identities': ['5dd68c3661ebec69bf92edb6'],
 'identity': '5dd68c3661ebec69bf92edb6',
 'platform': 'Android',
 'phone': 919148338065,
 'name': 'Veeresh H',
 'push_token': 'fcm:e8Mr0IJQvd4:APA91bEpYaTZb7EiAfnRZ5MEsFmWaSMLXHreR2_K_8p30GEBAfMm1KO2Rgbjiok9KVqToeEOfBoLrmhoT0Y7AHDPZ4F15g5BcvXCtbsmuBTIh_Xc0ZHMx-TS5FJbz9tF8EG6_0EPKt-i'}

[False, True, False, False, False, False]

In [42]:
list(profile_payload.values())[1]
for i,x  in enumerate(profile_payload):
    print(x)
    

'5dd68c3661ebec69bf92edb6'

all_identities
identity
platform
phone
name
push_token





  0%|          | 0/98964 [00:00<?, ?it/s]


 25%|██▌       | 25103/98964 [00:00<00:00, 251027.85it/s]


 59%|█████▉    | 58284/98964 [00:00<00:00, 270805.93it/s]


100%|██████████| 98964/98964 [00:00<00:00, 290306.66it/s]


In [67]:
order_timer_rendered_raw.shape

(98964, 7)

In [75]:
order_timer_rendered_raw[~order_timer_rendered_raw.captain_id.isna()]
order_timer_rendered_raw.iloc[327]['profile']
order_timer_rendered_raw.iloc[326]['profile']

,eventName,ts,eventProps,profile,deviceInfo,time,captain_id
0,orderTimerRendered,20200211103149,"{'area': '----', 'cluster': 'BRS nagar', 'city...",{'all_identities': ['randhirsingh55300@gmail.c...,"{'osVersion': '9', 'sdkVersion': '30601', 'mak...",2020-02-11 10:31:49,"5d2a01ae06ee2c37f1f6841e,5d71c3a69f130346e2c86cfb"
1,orderTimerRendered,20200211103151,"{'area': '----', 'cluster': '', 'city': 'Banga...",{'all_identities': ['5dd68c3661ebec69bf92edb6'...,"{'osVersion': '9', 'sdkVersion': '30601', 'mak...",2020-02-11 10:31:51,5dd68c3661ebec69bf92edb6
2,orderTimerRendered,20200211103151,"{'area': '----', 'cluster': '', 'city': 'Banga...",{'all_identities': ['5dd68c3661ebec69bf92edb6'...,"{'osVersion': '9', 'sdkVersion': '30601', 'mak...",2020-02-11 10:31:51,5dd68c3661ebec69bf92edb6
3,orderTimerRendered,20200211103155,"{'area': '----', 'cluster': 'Hex 3', 'city': '...",{'all_identities': ['5e0c51a5a10a71228f892a7c'...,"{'osVersion': '9', 'sdkVersion': '30601', 'mak...",2020-02-11 10:31:55,5e0c51a5a10a71228f892a7c
4,orderTimerRendered,20200211103153,"{'area': '----', 'cluster': 'SH-Rosalina', 'ci...",{'all_identities': ['5d3a9eb82f470b5bbc1fa69e'...,"{'osVersion': '8.0.0', 'sdkVersion': '30601', ...",2020-02-11 10:31:53,"5d3a9eb82f470b5bbc1fa69e,5d614bbba24c96105e22d83e"
...,...,...,...,...,...,...,...
98959,orderTimerRendered,20200211234246,"{'area': '----', 'cluster': 'BTM', 'city': 'Ba...",{'all_identities': ['5d19f75911e17032dce9505c'...,"{'osVersion': '8.1.0', 'sdkVersion': '30601', ...",2020-02-11 23:42:46,5d19f75911e17032dce9505c
98960,orderTimerRendered,20200211234256,"{'area': '----', 'cluster': 'BTM', 'city': 'Ba...",{'all_identities': ['5d19f75911e17032dce9505c'...,"{'osVersion': '8.1.0', 'sdkVersion': '30601', ...",2020-02-11 23:42:56,5d19f75911e17032dce9505c
98961,orderTimerRendered,20200211220509,"{'area': '----', 'cluster': 'Kishanpura', 'cit...",{'all_identities': ['5cc6a37fadc5f92951ee46c2'...,"{'osVersion': '8.0.0', 'sdkVersion': '30601', ...",2020-02-11 22:05:09,"5cc6a37fadc5f92951ee46c2,5d08b09a1c64cc470f61de8e"
98962,orderTimerRendered,20200211210331,"{'cluster': 'Mathikere', 'city': 'Bangalore', ...",{'all_identities': ['5ad1e59cf7284039532fbb42'...,"{'osVersion': '7.0', 'sdkVersion': '30601', 'm...",2020-02-11 21:03:31,5ad1e59cf7284039532fbb42


{'platform': 'Android',
 'phone': 919592389008,
 'name': 'Rajat Kumar',
 'push_token': 'fcm:dozSVanuEmw:APA91bEWr_Lxp_dSjVK4JIZSA6N_WS_Z3J9WswiTPi_Q8jFEzfkcUFH_x9JEj_hjppAxhAH0oFUFKKG8qvYEgwIHRTYXyuGfL013cI_LBqh7ZJIqlhFsdZ-EmrBSkj5JlKclyu3QdGcl'}

{'all_identities': ['594b664185992d305990c6f0',
  'ak23091990@gmail.com',
  '5be433ca246eb83dabc12404',
  'velanganni1958@gmail.com'],
 'identity': '594b664185992d305990c6f0,5be433ca246eb83dabc12404',
 'platform': 'Android',
 'phone': 919886712702,
 'name': 'vinay  S',
 'email': 'ak23091990@gmail.com',
 'push_token': 'fcm:ci9BOXKU6M8:APA91bFCnBHpCuy-FZ5NfWoLseoWVF4mTaB6hh-6YILRPIzQzliMraYby9x1OGBnpZH1yA5UgCNWwDGrYPs0xE4RQazaTisTuqAXbpG3VrJdXaYZFJkb0zgez-Xdagd14yBnYlR_kqe-'}

In [58]:
# print(a)
# import re
# m = re.search('[0-9a-fA-F]{10,}', a)
# if m:
#     found = m.group(0)
#     print(found) 

5dd68c3661ebec69bf92edb6


In [55]:
order_timer_rendered_raw['profile'].progress

0        {'all_identities': ['randhirsingh55300@gmail.c...
1        {'all_identities': ['5dd68c3661ebec69bf92edb6'...
2        {'all_identities': ['5dd68c3661ebec69bf92edb6'...
3        {'all_identities': ['5e0c51a5a10a71228f892a7c'...
4        {'all_identities': ['5d3a9eb82f470b5bbc1fa69e'...
                               ...                        
98959    {'all_identities': ['5d19f75911e17032dce9505c'...
98960    {'all_identities': ['5d19f75911e17032dce9505c'...
98961    {'all_identities': ['5cc6a37fadc5f92951ee46c2'...
98962    {'all_identities': ['5ad1e59cf7284039532fbb42'...
98963    {'all_identities': ['58cbbb3ed421dc596ed7f258'...
Name: profile, Length: 98964, dtype: object

In [57]:
order_timer_rendered_raw['profile'].iloc[0]

{'all_identities': ['randhirsingh55300@gmail.com',
  '5d2a01ae06ee2c37f1f6841e',
  '5d71c3a69f130346e2c86cfb'],
 'identity': '5d2a01ae06ee2c37f1f6841e,5d71c3a69f130346e2c86cfb',
 'platform': 'Android',
 'phone': 918968755300,
 'name': 'Barinder Singh',
 'email': 'randhirsingh55300@gmail.com',
 'push_token': 'fcm:d846nZvuiHs:APA91bGbUMMTTDn3CqFfZ7e6E38V9wXXzZ4bOThnxuXbvQgBXjKo_nED0HCRvZvzbqT2SzgFS6bYtj9KVjRJSDtPEbAGabJ5C_jduMWGJ282XHimjD40GhidDczLAXlrz3v4Vvu5gs6i'}

In [56]:
get_captain_identity(order_timer_rendered_raw['profile'].iloc[0])

'5d2a01ae06ee2c37f1f6841e,5d71c3a69f130346e2c86cfb'

In [53]:
get_captain_identity(order_timer_rendered_raw['profile'].iloc[2])

'5dd68c3661ebec69bf92edb6'

In [24]:
order_timer_rendered_raw['eventProps'].iloc[2]

{'area': '----',
 'cluster': '',
 'city': 'Bangalore',
 'Device rooted': 'false',
 'regCity': 'Bangalore',
 'CT Source': 'Mobile',
 'latitude': 12.711327,
 'longitude': 77.699585,
 'CT Latitude': 12.711327,
 'CT Longitude': 77.699585,
 'occurred_date': '$D_1581397311',
 'hhmmss': '103151',
 'orderId': '5e4235036735ce3fc338bc39',
 'yyyymmdd': '20200211',
 'networkspeed': '-1.0',
 'CT App Version': '3.10.2',
 'epoch': '1581397311990'}

In [23]:
order_timer_rendered_raw[['eventName','time']]

,eventName,time
0,orderTimerRendered,2020-02-11 10:31:49
1,orderTimerRendered,2020-02-11 10:31:51
2,orderTimerRendered,2020-02-11 10:31:51
3,orderTimerRendered,2020-02-11 10:31:55
4,orderTimerRendered,2020-02-11 10:31:53
...,...,...
98959,orderTimerRendered,2020-02-11 23:42:46
98960,orderTimerRendered,2020-02-11 23:42:56
98961,orderTimerRendered,2020-02-11 22:05:09
98962,orderTimerRendered,2020-02-11 21:03:31


In [17]:
test_time='20200211103149'

In [19]:
from datetime import datetime
test_time='20200211103149'
datetime_str = '09/19/18 13:55:26'


# datetime_object = datetime.strptime(test_time, '%m/%d/%y %H:%M:%S')

# print(type(datetime_object))
print(datetime_object)  # printed in default forma

2020-02-11 10:31:49
